In [5]:
# ISRO Satellite Image Change Detection - Colab Notebook
# Author: Nithish Chandra
# Purpose: Colab-ready workflow to train a U-Net model for land/water/vegetation segmentation
# and change detection between two timestamps.

# %% [markdown]
# # ISRO-ready: Satellite Image Change Detection (U-Net)
# Complete pipeline for training, evaluating, and visualizing segmentation and change detection on satellite images.

# %%
# === 1) Install required packages ===
!pip install --quiet tensorflow opencv-python matplotlib scikit-learn albumentations keras-unet-collection
!pip install --quiet kaggle

# %% [markdown]
# ## Kaggle dataset (optional)
# To download Kaggle datasets directly into Colab, upload kaggle.json or place it in your Drive under `MyDrive/kaggle/kaggle.json`.

# %%
import os
KAGGLE_PATH = '/root/.kaggle'
os.makedirs(KAGGLE_PATH, exist_ok=True)
if os.path.exists('/content/drive/MyDrive/kaggle/kaggle.json'):
    !cp /content/drive/MyDrive/kaggle/kaggle.json {KAGGLE_PATH}/kaggle.json
!chmod 600 {KAGGLE_PATH}/kaggle.json

# Example download (uncomment to use)
# !kaggle competitions download -c planet-understanding-the-amazon-from-space -p /content/dataset
# !unzip -q /content/dataset/planet-understanding-the-amazon-from-space.zip -d /content/dataset

# %% [markdown]
# ## Dataset structure
# Example folder setup for segmentation/change detection tasks.

# %%
import pathlib
BASE_DIR = '/content/dataset'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VAL_DIR = os.path.join(BASE_DIR, 'val')
CHECKPOINT_DIR = '/content/drive/MyDrive/isro_checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

IMG_SIZE = (256, 256)
BATCH_SIZE = 8
EPOCHS = 25
NUM_CLASSES = 3

# %%
import cv2
import numpy as np
from glob import glob
from tensorflow.keras.utils import Sequence

def read_image(path, size=IMG_SIZE):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    img = img.astype('float32') / 255.0
    return img

def read_mask(path, size=IMG_SIZE, num_classes=NUM_CLASSES):
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)
    one_hot = np.eye(num_classes)[mask]
    return one_hot.astype('float32')

class SegmentationSequence(Sequence):
    def __init__(self, image_pairs, mask_paths, batch_size=BATCH_SIZE, augment=None):
        self.image_pairs = image_pairs
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.augment = augment

    def __len__(self):
        return int(np.ceil(len(self.image_pairs) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_pairs = self.image_pairs[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_masks = self.mask_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        X, Y = [], []
        for (p1, p2), m in zip(batch_pairs, batch_masks):
            img1 = read_image(p1)
            img2 = read_image(p2)
            inp = np.concatenate([img1, img2], axis=-1)
            mask = read_mask(m)
            if self.augment:
                augmented = self.augment(image=inp, mask=mask)
                inp, mask = augmented['image'], augmented['mask']
            X.append(inp)
            Y.append(mask)
        return np.array(X), np.array(Y)

# %%
def find_pairs(images_dir):
    all_images = glob(os.path.join(images_dir, '*'))
    t1 = [p for p in all_images if '_t1' in os.path.basename(p)]
    pairs = []
    for p in t1:
        p2 = p.replace('_t1', '_t2')
        if os.path.exists(p2):
            pairs.append((p, p2))
    return sorted(pairs)

# train_pairs = find_pairs(os.path.join(TRAIN_DIR, 'images'))
# train_masks = sorted(glob(os.path.join(TRAIN_DIR, 'masks', '*')))
# val_pairs = find_pairs(os.path.join(VAL_DIR, 'images'))
# val_masks = sorted(glob(os.path.join(VAL_DIR, 'masks', '*')))

# %%
import albumentations as A
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=20, p=0.5),
])

# %%
import tensorflow as tf
from tensorflow.keras import layers, models

def get_unet(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 6), num_classes=NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)
    c1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2,2))(c1)

    c2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2,2))(c2)

    c3 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2,2))(c3)

    c4 = layers.Conv2D(512, (3,3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3,3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2,2))(c4)

    c5 = layers.Conv2D(1024, (3,3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3,3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(512, (2,2), strides=(2,2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3,3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3,3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(num_classes, (1,1), activation='softmax')(c9)
    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

model = get_unet()
model.summary()

# %%
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.reshape(y_true, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred, [-1, NUM_CLASSES])
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0) + smooth)
    return 1. - tf.reduce_mean(score)

def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1,2,3]) - intersection
    return tf.reduce_mean((intersection + smooth) / (union + smooth))

model.compile(optimizer='adam', loss=dice_loss, metrics=[iou_metric])

# %%
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
checkpoint_cb = ModelCheckpoint(os.path.join(CHECKPOINT_DIR, 'unet_best.h5'), save_best_only=True, monitor='val_iou_metric', mode='max')
reduce_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_cb = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# history = model.fit(train_seq, validation_data=val_seq, epochs=EPOCHS, callbacks=[checkpoint_cb, reduce_cb, early_cb])

# %%
import matplotlib.pyplot as plt
def visualize_prediction(model, img_t1_path, img_t2_path, mask_path=None):
    img1 = read_image(img_t1_path)
    img2 = read_image(img_t2_path)
    inp = np.concatenate([img1, img2], axis=-1)
    pred = model.predict(inp[np.newaxis, ...])[0]
    pred_class = np.argmax(pred, axis=-1)

    plt.figure(figsize=(12,6))
    plt.subplot(1,4,1); plt.imshow(img1); plt.title('T1'); plt.axis('off')
    plt.subplot(1,4,2); plt.imshow(img2); plt.title('T2'); plt.axis('off')
    plt.subplot(1,4,3); plt.imshow(pred_class); plt.title('Predicted Seg'); plt.axis('off')
    if mask_path:
        mask = np.argmax(read_mask(mask_path), axis=-1)
        plt.subplot(1,4,4); plt.imshow(mask); plt.title('Ground Truth'); plt.axis('off')
    plt.show()

# %%
def compute_change_map(pred_t1, pred_t2):
    change = (pred_t1 != pred_t2).astype(np.uint8)
    return change

# %%
# model.save('/content/drive/MyDrive/isro_checkpoints/unet_final.h5')


chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 6)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 256, 256,  │      3,520 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_39 (Conv2D)  │ (None, 256, 256,  │     36,928 │ conv2d_38[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 128, 128,  │          0 │ conv2d_39[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 128, 128,  │     73,856 │ max_pooling2d_8[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 128, 128,  │    147,584 │ conv2d_40[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 64, 64,    │          0 │ conv2d_41[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 64, 64,    │    295,168 │ max_pooling2d_9[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 64, 64,    │    590,080 │ conv2d_42[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_10    │ (None, 32, 32,    │          0 │ conv2d_43[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_10… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 32, 32,    │  2,359,808 │ conv2d_44[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 16, 16,    │          0 │ conv2d_45[0][0]   │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 16, 16,    │  4,719,616 │ max_pooling2d_11… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 16, 16,    │  9,438,208 │ conv2d_46[0][0]   │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_8  │ (None, 32, 32,    │  2,097,664 │ conv2d_47[0][0]   │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 32, 32,    │          0 │ conv2d_transpose

 Total params: 31,033,603 (118.38 MB)

 Trainable params: 31,033,603 (118.38 MB)

 Non-trainable params: 0 (0.00 B)